# Amazon SageMaker scikit-learn コンテナを使って NearestNeighbors の学習、推論を行う


---

このサンプルノートブックでは、SageMaker が用意した scikit-learn のコンテナを使って NearestNeighbors の学習と、学習したモデルを使った推論を行います。

まず初めに、SageMaker Python SDK を使ってジョブを起動する方法を紹介します。その後同様の操作を boto3 を使って行う方法を紹介します。

---
## セットアップ

必要なライブラリやパラメタを準備します。

In [ ]:
import os
import boto3
import re
import json
import pandas as pd
import numpy as np
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.model import SKLearnModel
from sklearn.datasets import fetch_california_housing

sagemaker_session = sagemaker.Session()
region = boto3.Session().region_name

role = get_execution_role()

project_name = 'sklearn-byo-script'
user_name = 'demo2'
bucket = sagemaker.Session().default_bucket()
prefix = f"sagemaker/{project_name}"

print(f"bucket: {bucket}")

## データの準備

このサンプルノートブックでは scikit-learn が用意したデータセットを使用します。

In [ ]:
from sklearn.datasets import load_wine

data = load_wine()
data

取得したデータには特徴量、ターゲット変数、メタデータなどが含まれるので、特徴量部分のみを抜き出して CSV ファイルにしてから S3 にアップロードします。

In [ ]:
import pandas as pd
pd.DataFrame(data['data']).to_csv('wine.csv', index=False, header=False)

In [ ]:
inputs = sagemaker_session.upload_data(path='wine.csv', bucket=bucket, key_prefix=f'{project_name}/{user_name}')
print('input spec (in this case, just an S3 path): {}'.format(inputs))

## SageMaker 学習ジョブを使って knn を学習する 

code/train ディレクトリの中の train.py を使って knn を学習します。code ディレクトリの中に requirements.txt があると、学習インスタンス起動時に SageMaker が自動的にそこに書かれたライブラリを pip install してくれます。

SageMaker Job は全てのジョブ名がユニークである必要があります。Scikit-learn 用の Estimator である SKLearn を作成する際に、パラメタ base_job_name にprefix を指定しておくと、SageMaker が自動的にタイムスタンプを付与します。フルでジョブ名を指定したい場合は fit() のパラメタ job_name を指定します。

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'train.py'

# run the Scikit-Learn script
sklearn = SKLearn(
    base_job_name=project_name+'-'+user_name,
    entry_point=script_path,
    source_dir='code/train',
    train_instance_type="ml.m5.large",
    role=role,
    framework_version='1.0-1',
    py_version='py3',
    sagemaker_session=sagemaker_session,
    hyperparameters={
        'n_neighbors': 2,
        'radius': 0.4
#         'metric': 'cosine'
    })
    
sklearn.fit({'train':inputs})

## 学習したモデルを使ってバッチ推論を実行する

SageMaker Processing を使ってバッチ推論を実行します。SageMaker Processing は、任意のコンテナとスクリプトを使ってジョブを柔軟に実行できる機能です。実行するスクリプトはオンプレミスなどのローカル環境で動作するものをほぼそのまま利用できます。[SageMaker バッチ変換](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/batch-transform.html) の機能でもバッチ推論は可能ですが、こちらは [SageMaker の仕様](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#serve-a-model) に則って推論スクリプトを書く必要があります。

以下のセルを実行して SageMaker が用意している Scikit-learn のコンテナイメージの URI を取得します。SageMaker のビルトインコンテナイメージは AWS が公開している Amazon ECR リポジトリに保存されています。

In [ ]:
from sagemaker.image_uris import retrieve

inference_repository_uri = retrieve(
    framework='sklearn',
    region=region,
    version='1.0-1',
    py_version='py3',
    instance_type='ml.m5.xlarge',
    
)
inference_repository_uri

In [ ]:
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput
from datetime import datetime
from dateutil import tz

JST = tz.gettz('Asia/Tokyo')
timestamp = datetime.now(JST).strftime('%Y%m%d-%H%M%S')

inference_job_name = project_name + '-' + user_name + '-' + timestamp
inference_input_data = inputs
inference_model = sklearn.model_data
inference_output_data = f's3://{bucket}/{prefix}/inference/{inference_job_name}'
code_path = '/opt/ml/processing/input/code'
input_dir = '/opt/ml/processing/input/data'
model_dir =  '/opt/ml/processing/input/model'
output_dir = '/opt/ml/processing/output'

SCRIPT_LOCATION = "code/inference"

code_s3_path = sagemaker_session.upload_data(
    SCRIPT_LOCATION,
    bucket=bucket,
    key_prefix=os.path.join(prefix, SCRIPT_LOCATION, timestamp),
)

# 入力データの設定
# バッチ推論をするために必要なソースコード、データ、モデルをインスタンスで使用するためのパスの設定
# source で指定した S3 パスから destination で指定したローカルパスに自動的にァイルがダウンロードされる
inference_inputs = [
    ProcessingInput(
        input_name='code',
        source=code_s3_path,
        destination=code_path
    ),
    ProcessingInput(
        input_name="data",
        source=inference_input_data,
        destination=input_dir
    ),
    ProcessingInput(
        input_name="model",
        source=inference_model,
        destination=model_dir
    )
]

# 出力データの設定
# 推論結果を保存するパスに関する設定
# source で指定したローカルパスから destination で指定した S3 パスに自動的にファイルがアップロードされる
inference_outputs = [
    ProcessingOutput(
        output_name="result",
        source=output_dir,
        destination=inference_output_data,
    )
]

inference_processor = Processor(
        role=role,
        image_uri=inference_repository_uri,
        entrypoint=["python3", f"{code_path}/inference.py"],
        instance_count=1, 
        instance_type="ml.m5.xlarge",
        volume_size_in_gb=16,
        volume_kms_key=None,
        output_kms_key=None,
        max_runtime_in_seconds=86400,  # default is 24 hours(60*60*24)
        sagemaker_session=None,
        env=None,
        network_config=None
    )

inference_processor.run(
    job_name=inference_job_name,
    inputs=inference_inputs,
     outputs=inference_outputs,
    arguments=['--n_neighbors', '2'],
    logs=False,
    wait=False
)
from IPython.display import display, Markdown
display(Markdown(f"<a href=\"https://s3.console.aws.amazon.com/s3/buckets/{bucket}?region={region}&prefix={prefix}/inference/{inference_job_name}/&showversions=false\" target=\"_blank\">推論結果 (S3)</a>"))


## boto3 を使ってジョブを実行する

ここまでは、SageMaker Python SDK を使って Training Job, Processing Job を実行しました。SageMaker Python SDK は非常に便利ですが、boto3 のみしか使えない環境では boto3 を使って同様のことが可能です。SageMaker Python SDK がよしなにやっていた部分を明示的に記述する必要がありますが、インフラ部分の細かな設定などは boto3 からのみ実行することができます。

### モデルの学習

学習ジョブで使用するソースコードは sourcedir.tar.gz という名前の tar.gz ファイルに圧縮して S3 にアップロードします。圧縮ファイル直下にファイルが配置されるようにしてください（ソースコードが入ったフォルダごと圧縮しない）。

In [ ]:
SCRIPT_LOCATION = "code/train"
TRAINNING_SCRIPT_LOCATION = "sourcedir.tar.gz"
!cd $SCRIPT_LOCATION && tar zcvf ../../$TRAINNING_SCRIPT_LOCATION ./*

train_code_s3_path = sagemaker_session.upload_data(
    TRAINNING_SCRIPT_LOCATION,
    bucket=bucket,
    key_prefix=os.path.join(prefix, SCRIPT_LOCATION, timestamp),
)
train_code_s3_path

create_training_job API を使って学習ジョブを実行します。ジョブの完了を待たずレスポンスが返ってきます。ジョブの完了は S3 バケットへの model.tar.gz の PutObject イベントで把握できます。ジョブ実行時に設定可能なパラメタについては [こちらのドキュメント](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_training_job) を参照してください。

In [ ]:
timestamp = datetime.now(JST).strftime('%Y%m%d-%H%M%S')
training_job_name = project_name + '-' + user_name + '-' + timestamp
output_location = f's3://{bucket}/{prefix}/train'

create_training_params = {
    "AlgorithmSpecification": {"TrainingImage": inference_repository_uri, "TrainingInputMode": "File"},
    "RoleArn": role,
    "OutputDataConfig": {"S3OutputPath": output_location},
    "ResourceConfig": {"InstanceCount": 1, "InstanceType": "ml.c4.xlarge", "VolumeSizeInGB": 50},
    "TrainingJobName": training_job_name,
    "HyperParameters": {
        'n_neighbors': "2",
        'radius': "0.4",
        'sagemaker_program' : "train.py",
        'sagemaker_submit_directory': train_code_s3_path
    },
    "StoppingCondition": {"MaxRuntimeInSeconds": 60 * 60},
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": inputs,
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None",
        }
    ],
}


sagemaker_client = boto3.client("sagemaker")

sagemaker_client.create_training_job(**create_training_params)

### バッチ推論

create_processing_job API も create_training_job API と同様に実行後すぐにレスポンスが返ってきます。ジョブ実行時に設定可能なパラメタについては [こちらのドキュメント](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_processing_job) を参照してください。

In [ ]:
timestamp = datetime.now(JST).strftime('%Y%m%d-%H%M%S')

inference_job_name = project_name + '-' + user_name + '-' + timestamp
inference_input_data = inputs
inference_model = f'{output_location}/{training_job_name}/output/model.tar.gz'
inference_output_data = f's3://{bucket}/{prefix}/inference/{inference_job_name}'

sagemaker_client.create_processing_job(
    ProcessingInputs=[
        {
            'InputName': 'code',
            'S3Input': {
                'S3Uri': code_s3_path,
                'LocalPath': code_path,
                'S3DataType': 'S3Prefix',
                'S3InputMode': 'File',
            },
        },
        {
            'InputName': 'data',
            'S3Input': {
                'S3Uri': inference_input_data,
                'LocalPath': input_dir,
                'S3DataType': 'S3Prefix',
                'S3InputMode': 'File'
            },
        },
        {
            'InputName': 'model',
            'S3Input': {
                'S3Uri': inference_model,
                'LocalPath': model_dir,
                'S3DataType': 'S3Prefix',
                'S3InputMode': 'File'
            },
        },
    ],
    ProcessingOutputConfig={
        'Outputs': [
            {
                'OutputName': 'result',
                'S3Output': {
                    'S3Uri': inference_output_data,
                    'LocalPath': output_dir,
                    'S3UploadMode': 'EndOfJob'
                },
            },
        ]
    },
    ProcessingJobName=inference_job_name,
    ProcessingResources={
        'ClusterConfig': {
            'InstanceCount': 1,
            'InstanceType': 'ml.m5.xlarge',
            'VolumeSizeInGB': 16,
        }
    },
    AppSpecification={
        'ImageUri': inference_repository_uri,
        'ContainerEntrypoint': [
            'python3',
            f'{code_path}/inference.py',
        ],
        'ContainerArguments': [
            '--n_neighbors', '2'
        ]
    },
    RoleArn=role,
)